### 연습문제 
- Doc2Vec 라이브러리 이용한 감정에 분석 
- 데이터는 ratings_train.txt파일을 로드
- 결측치 제거
- 특수 문자, 2칸 이상의 공백의 문자를 제거하는 문자열 좌우 공백 제거 정규화 함수 
- document 컬럼의 데이터에서 중복 데이터를 제거 
- 빈 텍스트, " "가 존재한다면 해당 행 데이터도 제거 
- 상위의 5000개 정도 데이터를 이용 (14만 개는 시간이 너무 오래 걸림)
- 토큰화 함수 (Komoran를 이용)
    - 필요한 품사 : NNP, NNG, VV, VA, MAG, XR 만을 사용
    - 불용어 단어 : 하다, 되다, 이다, 것 , 수, 거 단어들은 제외
- 데이터에서 독립(document) , 종속(label) 변수로 데이터를 나눠주고 train, test 데이터셋 분할 (비율은 8:2)
- Doc2Vec 객체를 생성하여 학습 
    - 매개변수 
        - vector_size = 200
        - window = 5
        - min_count = 2
        - dm = 1
        - negative = 5
        - seed = 42
        - epochs = 50
    - 학습 시키는 데이터는 X_train
- X_train, X_test -> 문자열 데이터 -> infer_vector() 함수를 이용해서 임베딩  
- 고전 머신러닝 분류 모델을 이용하여 입베딩된 데이터를 독립 변수로 Y의 데이터들을 종속 변수로 학습으로 예측
    - 정확도를 확인 
    - LogisticRegression(max_iter = 2000, random_state = 42)
    - LinearSVC(random_state = 42)
    - 두개의 모델을 사용하여 정확도가 좋은 모델을 선택

In [1]:
import pandas as pd
import re
import numpy as np
from konlpy.tag import Komoran
from sklearn.model_selection import train_test_split
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

In [114]:
# !pip install tqdm

In [2]:
#  진행 상황들을 로그에 표시해주는 라이브러리
from tqdm import tqdm

In [3]:
# 데이터 로드
df = pd.read_csv("../data/ratings_train.txt", sep='\t')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [4]:
# 결측치 확인
df[df['document'].isna()]

,id,document,label
25857,2172111,NaN,1
55737,6369843,NaN,1
110014,1034280,NaN,0
126782,5942978,NaN,0
140721,1034283,NaN,0


In [5]:
# 총 15만 개 중 결측치는 5개이므로 전체에 비해 매우 적기 때문에 제거
df.dropna(inplace=True)
df.isna().sum()

id          0
document    0
label       0
dtype: int64

---
#### cf. 실수하기 쉬운 케이스
- **case 1.**
    ```python
    df['document'] = df['document'].dropna()
    ```
    이렇게 하면 ```df.isna().sum()```으로 확인했을 때 여전히 5개의 결측치가 남아 있다.
    ```python
    df['document']            # 인덱스 150,000개
    df['document'].dropna()   # 인덱스 149,995개

    ⇒ 문제는 이 짧아진 Series를 다시 원본 데이터프레임의 길이가 그대로인 df['document'] 컬럼에 할당할 때 발생한다.
    Pandas는 Series를 다른 Series나 DataFrame 컬럼에 할당할 때, 인덱스를 기준으로 정렬하고 매칭을 시도한다.
    따라서 dropna()를 통해 제거되었던 인덱스들은 새로운 Series(df['document'].dropna())에는 포함되어 있지 않다. 이 인덱스들에 해당하는 위치에 할당할 값이 없으므로, Pandas는 이 위치를 다시 결측치(NaN)로 채운다.

    ⇒ 컬럼의 결측치를 제거하려면, 해당 결측치를 포함하는 행 전체를 제거해야 한다.
        - 방법 1: dropna()를 행 전체에 적용 (권장)
            # 'document' 컬럼에 NaN이 있는 행 전체를 삭제
            df.dropna(subset=['document'], inplace=True)
        - 방법 2: 결측치를 빈 문자열('')로 대체 (Komoran 오류 방지에도 유용)
            # 결측치를 빈 문자열로 대체하여 행의 길이를 유지합니다.
            df['document'] = df['document'].fillna('')
- **case 2.** 컬럼을 데이터프레임으로 저장
    ```python
    df = df['document'].dropna()
    ```
---

In [6]:
# 텍스트 정규화 함수
def normalize(text):
    text = re.sub(r"[^가-힣0-9a-zA-Z\s\.]", " ", str(text))
    text = re.sub(r"\s+", " ", text).strip()
    return text

# 혹시 모를 TypeError 방지를 위해 str(text)
# text 매개변수에 들어오는 데이터는 df['document']의 values (리뷰 데이터들)
# 리뷰 데이터가 문자가 아닌 경우 문자형으로 변경

In [7]:
df = df.applymap(normalize)

C:\Users\student\AppData\Local\Temp\ipykernel_816\614872782.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(normalize)


---
#### cf.
```python
df['document']에서 normalize 함수를 이용
normalize에 들어가는 인자는 문자열이 기본
normalize(df['document'])는 잘못된 부분
```
```apply()``` 함수는 데이터프레임에서 사용하는 함수(2차원에서 1차원을 뽑아주는 함수)로,
</br> ```map()``` 함수와 비슷한 기능. 각각의 원소들을 추출하여 어떤 작업(함수)들을 함.

In [8]:
df.head(3).map(lambda x: print(x))      # value들을 하나씩 뽑아줌

9976970
3819312
10265843
아 더빙.. 진짜 짜증나네요 목소리
흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
너무재밓었다그래서보는것을추천한다
0
1
0


,id,document,label
0,None,None,None
1,None,None,None
2,None,None,None


In [9]:
df.head(3).apply(lambda x: str(x), axis=0)      # axis=0 이 기본값
# type(df.head(3).apply(lambda x: str(x)))      # Series

id          0     9976970\n1     3819312\n2    10265843\nN...
document    0                  아 더빙.. 진짜 짜증나네요 목소리\n1    흠...
label       0    0\n1    1\n2    0\nName: label, dtype: ob...
dtype: object

In [10]:
# 참고
df.head(3).apply(lambda x: x)
# str을 빼면 Series + Series + Series 이므로 데이터프레임으로 출력

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [11]:
# apply()를 이용해 map()처럼 원소를 하나하나 뽑아주고 싶다면
df.head(3).applymap(lambda x: print(x))

9976970
3819312
10265843
아 더빙.. 진짜 짜증나네요 목소리
흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
너무재밓었다그래서보는것을추천한다
0
1
0


C:\Users\student\AppData\Local\Temp\ipykernel_816\740592596.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df.head(3).applymap(lambda x: print(x))


,id,document,label
0,None,None,None
1,None,None,None
2,None,None,None


---

In [12]:
# 빈 텍스트(" "), 공백 텍스트(길이 1 이하) 있는지 확인
df.loc[ df['document'].isin( ["", " "] ) ]

,id,document,label
972,7425748,,0
1840,7095375,,1
2159,7070900,,1
2504,7449459,,0
2648,423224,,1
...,...,...,...
148560,7903625,,0
148940,6907774,,0
149364,8014701,,1
149398,7432786,,0


In [13]:
# 빈 텍스트(" "), 공백 텍스트(길이 1 이하) 제거
df = df.loc[ df['document'].str.len() > 1 ]

In [14]:
# document 컬럼의 중복 데이터 제거
df = df.drop_duplicates('document')

- 토큰화 함수 생성 (Komoran를 이용)
    - 필요한 품사 : NNP, NNG, VV, VA, MAG, XR
    - 불용어 단어 : 하다, 되다, 이다, 것 , 수, 거

In [15]:
komoran = Komoran()

allow_pos = ['NNP', 'NNG', 'VV', 'VA', 'MAG', 'XR']
stop_word = ['하다', '되다', '이다', '것', '수', '거']

def tokenize(text):
    tokens = []
    for word, pos in komoran.pos(text):
        if word not in stop_word and pos in allow_pos:
            tokens.append(word)
    return tokens

In [16]:
df = df.head(5000).copy()

In [17]:
# 토큰화된 리스트를 데이터프레임의 새로운 컬럼에 추가
tokenized_sentence = [ tokenize(val) for val in df['document'].values ]
tokenized_sentence

[['더빙', '진짜', '짜증', '나', '목소리'],
 ['포스터', '초딩', '영화', '오버', '연기', '가볍'],
 [],
 ['교도소', '이야기', '솔직히', '재미', '없', '평점', '조정'],
 ['익살', '연기', '돋보이', '영화', '스파이더맨', '늙', '보이', '하', '커스틴 던스트', '너무나'],
 ['막', '걸음마', '떼', '초등학교', '학년', '용', '영화', '별', '반개', '아깝'],
 ['원작', '긴장감', '제대로', '살리'],
 ['반개',
  '아깝',
  '욕',
  '나오',
  '이응경',
  '길용우',
  '연기',
  '생활',
  '이',
  '정말',
  '발로',
  '납치',
  '감금',
  '반복',
  '반복',
  '이',
  '드라마',
  '가족',
  '없',
  '연기',
  '못하',
  '사람',
  '모이'],
 ['액션', '없', '재미', '있', '안', '영화'],
 ['왜', '평점', '낮', '꽤', '보', '헐리우드', '화려', '너무', '길들이', '있'],
 [],
 ['볼', '때', '눈물', '나서', '죽', '향수', '자극', '허진호', '감성', '절제', '멜로', '달인'],
 ['울', '손들', '횡단보도', '건너', '때', '뛰쳐나오', '이범수', '연기', '드럽'],
 ['담백', '깔끔', '좋', '신문', '기사', '로만', '보다', '보', '자꾸', '잊어버리', '사람'],
 ['취향',
  '존중',
  '진짜',
  '극장',
  '보',
  '영화',
  '가장',
  '노',
  '재',
  '노',
  '감동',
  '스토리',
  '어거지',
  '감동',
  '어거지'],
 ['매번', '긴장'],
 ['참',
  '사람',
  '웃기',
  '바스코',
  '이기',
  '락스',
  '코',
  '까',
  '고',
  '바비',
  '이기',
  '아이

In [18]:
df['document_token'] = tokenized_sentence
df.head()

,id,document,label,document_token
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0,"[더빙, 진짜, 짜증, 나, 목소리]"
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1,"[포스터, 초딩, 영화, 오버, 연기, 가볍]"
2,10265843,너무재밓었다그래서보는것을추천한다,0,[]
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0,"[교도소, 이야기, 솔직히, 재미, 없, 평점, 조정]"
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,"[익살, 연기, 돋보이, 영화, 스파이더맨, 늙, 보이, 하, 커스틴 던스트, 너무나]"


df['document']에 넣지 않고 새로운 컬럼을 생성해 넣은 이유
</br> : 추후 확인을 위해 원본 데이터를 유지

In [ ]:
# 토큰화된 문서에 Tag를 부착
def tagged_docs(tokenized_data):
    tagged = []
    for d_id, toks in enumerate(tokenized_data):
        # toks의 길이가 0이라면 학습에서 의미없는 문장
        if len(toks) == 0:
            continue
        tagged.append( TaggedDocument( words=toks, tags=[f'DOC_{d_id}'] ) )
        # tags = f'DOC_{d_id}' 를 []로 묶어주지 않으면 단어 하나하나로 인식해
        # len(model.dv)이 14로 출력됨
    return tagged

- 데이터에서 독립(document) , 종속(label) 변수로 데이터를 나눠주고 train, test 데이터셋 분할 (비율은 8:2)

In [20]:
train_df, test_df = train_test_split(
    df, test_size= 0.2, random_state= 42, stratify=df['label']
)
# stratify=df['label']
# 라벨 컬럼에 따라, label 값이 0인 값들 중 따로 train, test를 나눠주고
#   1인 값들 중 따로 train, test를 나눠줌

In [21]:
train_df.shape

(4000, 4)

In [22]:
train_df['label'].value_counts()

label
0    2007
1    1993
Name: count, dtype: int64

In [35]:
# Doc2Vec에서 사용할 데이터는 train_df의 document_token 컬럼의 데이터
tagged_train = tagged_docs(train_df['document_token'].values)

In [24]:
len(tagged_train)

3925

- Doc2Vec 객체를 생성하여 학습 
    - 매개변수 
        - vector_size = 200
        - window = 5
        - min_count = 2
        - dm = 1
        - negative = 5
        - seed = 42
        - epochs = 50
    - 학습 시키는 데이터는 X_train

In [36]:
# 모델 자동 학습
model = Doc2Vec(
    documents= tagged_train,
    vector_size= 200,
    window= 5,
    min_count= 2,
    dm= 1,
    negative= 5,
    seed= 42,
    epochs= 50
)

In [26]:
# 모델 수동 학습
model2 = Doc2Vec(
    vector_size= 200,
    window= 5,
    min_count= 2,
    dm= 1,
    negative= 5,
    seed= 42,
    epochs= 50
)

# document가 미리 존재하는지의 여부에 따라 달라짐

# 단어 사전 생성
model2.build_vocab(tagged_train)
model2.train(tagged_train, total_examples=len(tagged_train), epochs=50)

In [27]:
# 단어 사전의 개수 확인
# 단어별 임베딩 벡터 wv 이용
print("단어 사전의 개수: ", len(model.wv))
print("단어 사전의 개수: ", len(model2.wv))

단어 사전의 개수:  2827
단어 사전의 개수:  2827


In [28]:
len(model.dv)

14

In [29]:
# 새로운 문장을 model에 infer_vector 함수를 이용하여 임베딩
def infer_vectors(model, norm_texts, epochs=50):
    # norm_tests: 정규화 처리가 끝난 문서들
    # model: 임베딩 모델
    vecs = []

    for text in norm_texts:
        # text: norm_texts의 각 원소들을 대입
        tokens = tokenize(text)
        # 토큰화된 데이터의 길이가 0인 경우
        if len(tokens) == 0:
            # 비어있는 토큰 데이터는 인덱스의 개수를 맞추기 위해서 영벡터 or 평균 벡터로 대체 가능
            # 영벡터로 출력
            vecs.append( np.zeros(model.vector_size, dtype=np.float32) )
            # 이 과정을 생략하면 ***
        else:
            vec = model.infer_vector(tokens, epochs=epochs)
            vecs.append(vec)
    return np.vstack(vecs)

In [30]:
X_train = infer_vectors( model, train_df['document'].values)
Y_train = train_df['label'].values

X_test = infer_vectors( model, test_df['document'].values)
Y_test = test_df['label'].values

In [31]:
print(X_train.shape)
print(Y_train.shape)

print(X_test.shape)
print(Y_test.shape)

(4000, 200)
(4000,)
(1000, 200)
(1000,)


In [32]:
# 로지스틱 회귀 모델과 선형 서포트벡터 분류 모델에 학습하고 정확도를 체크하는 함수
def eval_clf(clf, X_tr, Y_tr, X_te, Y_te, model_name):
    # clf: 분류 모델
    # tr: train 데이터
    # te: test 데이터
    # model_name: print에서 어떤 모델을 사용했는지 볼 수 있도록
    # 모델에 학습
    clf.fit(X_tr, Y_tr)
    # 모델을 통한 예측
    pred = clf.predict(X_te)
    # 정확도
    acc = accuracy_score(Y_te, pred)
    print(f"{model_name} 모델의 예측 정확도: {acc*100 :.1f}%")
    # class report
    clf_report = classification_report(Y_te, pred)
    print(f"{model_name} 모델의 report: \n {clf_report}")


In [33]:
# 모델 생성
logi = LogisticRegression(max_iter=2000, random_state=42)
eval_clf(logi, X_train, Y_train, X_test, Y_test, "LogisticRegression")

svc = LinearSVC(random_state=42)
eval_clf(svc, X_train, Y_train, X_test, Y_test, "LinearSVC")

LogisticRegression 모델의 예측 정확도: 71.2%
LogisticRegression 모델의 report: 
               precision    recall  f1-score   support

           0       0.73      0.67      0.70       502
           1       0.70      0.75      0.72       498

    accuracy                           0.71      1000
   macro avg       0.71      0.71      0.71      1000
weighted avg       0.71      0.71      0.71      1000

LinearSVC 모델의 예측 정확도: 71.9%
LinearSVC 모델의 report: 
               precision    recall  f1-score   support

           0       0.73      0.69      0.71       502
           1       0.71      0.75      0.73       498

    accuracy                           0.72      1000
   macro avg       0.72      0.72      0.72      1000
weighted avg       0.72      0.72      0.72      1000

